In [209]:
# dependancies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
from random import randint
from time import sleep
from config2 import password

In [2]:
# set url and get page data
url = 'https://locations.sonicdrivein.com/tx.html'
response = requests.get(url)

In [214]:
# Collect city paths
soup = BeautifulSoup(response.text, 'html.parser')

locations = soup.body.find_all('a', class_='c-directory-list-content-item-link')

city_urls = []
for location in locations:
    path = "https://locations.sonicdrivein.com/" + location['href']
    city_urls.append(path)

print("Scraping Complete")
city_urls

Scraping Complete


[]

In [4]:
n = len(city_urls)
n

474

In [5]:
Brand = []
Addresses = []
Localities = []
States = []
Postal_codes = []
Date = []
Time = []

for x in city_urls[:]:
    response = requests.get(x)
    soup3 = BeautifulSoup(response.text, 'html.parser')
# Address collection   
    units2 = soup3.find_all('span', class_='c-address-street-1')
    for b in units2:
        Brand.append("Sonic Drive-In")
        name2 = b.text
        Addresses.append(name2)
# locality collection    
    units3 = soup3.find_all('span', class_='c-address-city')
    for c in units3:
        name3 = c.text
        Localities.append(name3)
# State collection    
    units4 = soup3.find_all('abbr', class_='c-address-state')
    for d in units4:
        name4 = d.text
        States.append(name4)
# Postal Code collection    
    units5 = soup3.find_all('span', class_='c-address-postal-code')
    for e in units5:
        name5 = e.text
        Postal_codes.append(name5)
# Add time stamp
        current_date = datetime.datetime.now()
        Date.append(current_date.strftime("%Y-%m-%d"))
        Time.append(current_date.strftime("%H:%M:%S"))
    #sleep(randint(1,4))
        
print("Scraping Complete")

Scraping Complete


In [6]:
len(Addresses)

2334

In [169]:
# Put data into a dataframe
Location_data = pd.DataFrame(np.column_stack([Brand, Addresses, Localities, States, Postal_codes, Date, Time]),\
                             columns=["Brand", "Address", "City", "State", "Postal Code", "Date", "Time"])

# Clean data: sort, drop duplicates, set index
Location_data.sort_values(["State", "City","Address"], inplace=True)
Location_data.drop_duplicates(subset ="Address", keep="first", inplace=True) 
Location_data.set_index('Brand',inplace=True, drop=True)
#remove common after City
Location_data.loc[:, 'City'] = Location_data['City'].str[:-1]
# Need - drop State rows to include Texas only
Location_data = Location_data[Location_data.State == 'TX']

# Describe and view head
Sonic_df = Location_data
Sonic_df.head()

,Address,City,State,Postal Code,Date,Time
Brand,,,,,,
Sonic Drive-In,1442 Barrow Street,Abilene,TX,79605,2019-02-21,19:33:29
Sonic Drive-In,1734 N. First Street,Abilene,TX,79603,2019-02-21,19:33:29
Sonic Drive-In,202 N. Judge Ely Boulevard,Abilene,TX,79601,2019-02-21,19:33:29
Sonic Drive-In,2402 Sayles Boulevard,Abilene,TX,79605,2019-02-21,19:33:29
Sonic Drive-In,3856 S. Clack Street,Abilene,TX,79606,2019-02-21,19:33:29


In [170]:
Wendys_df = pd.read_csv("C:/Users/May 2018/Desktop/SMU Bootcamp Doc/ETL_Project/Wendys_Texas_Locations_Detailed_Version.csv")
Wendys_df.set_index('Brand',inplace=True, drop=True)
Wendys_df.head()

,Address,City,State,Postal Code,Date,Time
Brand,,,,,,
Wendy's,4374 Southwest Dr,Abilene,TX,79606,2019-02-20,11:17:23
Wendy's,3710 Beltline Road,Addison,TX,75001,2019-02-20,11:17:23
Wendy's,2025 W McDermott Drive,Allen,TX,75013,2019-02-20,11:17:24
Wendy's,601 W McDermott Drive,Allen,TX,75013,2019-02-20,11:17:24
Wendy's,2000 Ross-Osage St,Amarillo,TX,79103,2019-02-20,11:17:24


In [195]:
merged = pd.concat([Sonic_df, Wendys_df], )
len(merged)

1371

In [87]:
Wendys_count = Wendys_df["Postal Code"].count()
Sonic_count = Sonic_df["Postal Code"].count()
print(Wendys_count)
print(Sonic_count)

399
972


In [103]:
Sonic_df_nums = pd.to_numeric(Sonic_df['Postal Code'])

Sonic_zip_list = list(Sonic_df_nums)
Wendys_zip_list = list(Wendys_df["Postal Code"])

shared_zip_codes1 = 0

for zip in Wendys_zip_list:
    shared_zip_codes1 += Sonic_zip_list.count(zip)
print(shared_zip_codes1)

239
239


In [108]:
Wendys_overlap = shared_zip_codes1/Wendys_count * 100
Sonic_overlap = shared_zip_codes1/Sonic_count * 100

print(f'Wendys Overlap %: {Wendys_overlap}, Sonic Overlap %: {Sonic_overlap}')

Wendys Overlap %: 59.899749373433586, Sonic Overlap %: 24.588477366255145


In [171]:
#Dependencies
import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# PyMySQL
import pymysql
import pymysql.cursors
pymysql.install_as_MySQLdb()

#config file
from config import password

In [210]:
start = "mysql://root:"
end = "@localhost:3306/etl_project_db"
path = start + password + end
engine = create_engine(path)
#"mysql://root: @localhost:3306/etl_project_db"

In [211]:
Sonic_df.to_sql(name='sonic', con=engine, if_exists='replace', index=False)
Wendys_df.to_sql(name='wendys', con=engine, if_exists='replace', index=False)
merged.to_sql(name='merged', con=engine, if_exists='replace', index=False)

In [185]:
sonic = engine.execute("SELECT * FROM sonic").fetchall()
sonic = pd.DataFrame(sonic)
sonic.head()

,0,1,2,3,4,5
0,1442 Barrow Street,Abilene,TX,79605,2019-02-21,19:33:29
1,1734 N. First Street,Abilene,TX,79603,2019-02-21,19:33:29
2,202 N. Judge Ely Boulevard,Abilene,TX,79601,2019-02-21,19:33:29
3,2402 Sayles Boulevard,Abilene,TX,79605,2019-02-21,19:33:29
4,3856 S. Clack Street,Abilene,TX,79606,2019-02-21,19:33:29


In [186]:
wendys = engine.execute("SELECT * FROM wendys").fetchall()
wendys = pd.DataFrame(wendys)
wendys.head()

,0,1,2,3,4,5
0,4374 Southwest Dr,Abilene,TX,79606,2019-02-20,11:17:23
1,3710 Beltline Road,Addison,TX,75001,2019-02-20,11:17:23
2,2025 W McDermott Drive,Allen,TX,75013,2019-02-20,11:17:24
3,601 W McDermott Drive,Allen,TX,75013,2019-02-20,11:17:24
4,2000 Ross-Osage St,Amarillo,TX,79103,2019-02-20,11:17:24


In [212]:
merged2 = engine.execute("SELECT * FROM merged").fetchall()
merged2 = pd.DataFrame(merged2)
len(merged2)
merged2.head()

,0,1,2,3,4,5
0,1442 Barrow Street,Abilene,TX,79605,2019-02-21,19:33:29
1,1734 N. First Street,Abilene,TX,79603,2019-02-21,19:33:29
2,202 N. Judge Ely Boulevard,Abilene,TX,79601,2019-02-21,19:33:29
3,2402 Sayles Boulevard,Abilene,TX,79605,2019-02-21,19:33:29
4,3856 S. Clack Street,Abilene,TX,79606,2019-02-21,19:33:29
